# Will This Get Me Cancelled?

## Imports

In [177]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier, Perceptron, SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import pickle


## Classes and Functions

In [2]:
class Messages:
    def __init__(self, tweet, hater):
        self.given_text = tweet
        self.label = hater
        self.is_hate = self.is_hater()

    def is_hater(self):
        if self.label == 'No':
            return False
        else:
            return True
        
class MessageContainer:
    def __init__(self, message):
        self.message = message
        
    def get_text(self):
        return [x.given_text for x in self.message]
    
    def get_sentiment(self):
        return [x.is_hate for x in self.message]
        
    def evenly_distribute(self):
        positive = list(filter(lambda x: x.is_hate == False, self.message))
        hate = list(filter(lambda x: x.is_hate == True, self.message))
        positive_shrunk = positive[:len(hate)]
        self.reviews = hate + positive_shrunk
        random.shuffle(self.reviews)
    
    def len(self):
        return len(self.message)
    

def SetCreator(arr, arr2, param):
    if param == 'text':
        for x in range(0, arr2.len()):
            buffer = arr2.get_text()[x]
            arr.append(buffer)
    elif param == 'hate':
        for x in range(0, arr2.len()):
            buffer = arr2.get_sentiment()[x]
            arr.append(buffer)

def Categoriser(df, type):
    for x in df.index:
        if type in df['text'][x]:
            print(df['text'][x])

def predictor(text, model):
    text_v = vectorizer.transform([text])

    print(model.predict(text_v))

## Data Preparation

In [3]:
df = pd.read_csv('data_set\labeled_data3.csv')

message_container = []

for x in df.index:
    message = Messages(df['text'][x],df['label'][x])
    message_container.append(message)


train, test = train_test_split(message_container, test_size=0.2)

train_container = MessageContainer(train)
test_container = MessageContainer(test)

train_container.evenly_distribute()

train_x = []
train_y = []

test_x = []
test_y = []

SetCreator(train_x, train_container, param='text')
SetCreator(train_y, train_container, param='hate')

SetCreator(test_x, test_container, param='text')
SetCreator(test_y, test_container, param='hate')


## Vectorisation

In [178]:
vectorizer = HashingVectorizer()

train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)


## Support Vector Model

#### Training

In [199]:
#clf_rfm = RandomForestClassifier()

#clf_svm = svm.SVC(kernel="rbf", C=32)

#clf_log = LogisticRegression(max_iter=100000)

#clf_log.fit(train_x_vectors, train_y)

#clf_rfm.fit(train_x_vectors, train_y)

#clf_svm.fit(train_x_vectors, train_y)

clf_par = PassiveAggressiveClassifier()

clf_par.fit(train_x_vectors, train_y)

clf_per = Perceptron()

clf_per.fit(train_x_vectors, train_y)

models = [clf_par, clf_per]



In [197]:
clf_per.partial_fit(train_x_vectors, train_y)
clf_par.partial_fit(train_x_vectors, train_y)

PassiveAggressiveClassifier()

In [164]:
with open('./models/sgd_hate_classifier.pkl', 'rb') as f:
    huji = pickle.load(f)

models.append(huji)

In [200]:
for x in models:
    print(x)
    print(x.score(test_x_vectors, test_y))
    print(f1_score(test_y, x.predict(test_x_vectors), average=None, labels=[True, False]))
    print('\n')

PassiveAggressiveClassifier()
0.7112403100775194
[0.56047198 0.78499278]


Perceptron()
0.7073643410852714
[0.57464789 0.77695716]




#### Testing

In [201]:
model = clf_per

print(model.score(test_x_vectors, test_y))

f1_score(test_y, model.predict(test_x_vectors), average=None, labels=[True, False])

0.7073643410852714


array([0.57464789, 0.77695716])

## Pickling

In [176]:
with open('./models/per_hate_classifier.pkl', 'wb') as f:
    pickle.dump(model, f)

In [175]:
with open('./models/vectorizer.pk', 'wb') as fin:
    pickle.dump(vectorizer, fin)